# Optimisation
## Linear, Mixed Integer, Linear Constrained Optimisation with Investment applications


## Contents
* [Linear Programming](#first-bullet)
 - [Marketing Budget: Returns per country per product - Resource Allocation with Low Uncertainty](#low-uncertainty)
 - [Investment Budget: Return vs Risk - Resource Allocation with Low Uncertainty](#low-uncertainty)
 - [Stock Levels Problem - Oil Mixing](#oil-bullet)
* [Integer and Mixed Integer Linear Programming](#second-bullet)
 - [Knapsack Problem - Cargo consignment](#knapsack-bullet)
 - [Goal Programming/Multiple Objectives](#goals-bullet)
 - [Travelling Salesman](#TSP)
 - [Machine Scheduling](#MSP)

# Linear Programming <a class="anchor" id="first-bullet"></a>

In [ ]:
# package installations
!pip install -q pyomo
!apt-get install -y -qq glpk-utils
#SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model).write()

     |████████████████████████████████| 9.7 MB 4.6 MB/s 
     |████████████████████████████████| 49 kB 6.2 MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 123934 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up 

### **Marketing Budget** - Resource Allocation with Low Uncertainty <a class="anchor" id="[](low-uncertainty)"></a>

This problem shows a question of how to allocate a company's marketing budget wth respect to multiple countries and products that the company offers. The premise is that the company wishes to invest in new countries where the company has little market share, and wishes to increase it. The company has an idea of the response of the country to each product that the company sells in terms of revenue earned per revenue spent in marketing. This is a Linear Programming problem.

The objective of the problem is to maximise revenue due to the marketing, while adhering to certain criteria on the minimum amount of revenue generated per country, and the relative revenue of the Enhanced product to the Standard product

**Problem parameters and variables**
*   Marketing budget - £195M
*   Countries marketed to - India, China and Brazil
*   Products advertised - Standard and Enhanced
*   Country responses (£/£) to Standard product - 0.05, 0.04, 0.03
*   Country responses (£/£) to Enhanced product - 0.02, 0.03, 0.0275
*   Minimum Revenue per Country - £3M, £4M, £2M 
*   Enhanced Revenue/Standard Revenue - At Least 70%   

In [ ]:
# Model inputs for revenue, costs, and constraint parameters

# Costs
# Make the price of unobtainable oils sufficiently high such that the amount
# of these oils purchased at this time is forced to be zero for an optmum solution
import numpy as np
country_index = ['india','china','brazil']
product_index = ['standard','enhanced']

marketing_budget = 250.0
enhanced_to_standard_revenue_ratio = 0.7 
country_revenue_minima = [3.0, 4.0, 2.0]

response_factors = np.array([[0.05, 0.04, 0.03],
                             [0.02, 0.03, 0.0275]])

#### Marketing Budget Problem Code

In [ ]:
# general imports
import numpy as np

# import pyomo framework
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [ ]:
# make model variables, of each type(used/bought/stored), for each oil, for each month
model = pyo.ConcreteModel()
model.standard_india = pyo.Var(domain = pyo.Reals, bounds = (0,None))
model.enhanced_india = pyo.Var(domain = pyo.Reals, bounds = (0,None))
model.standard_china = pyo.Var(domain = pyo.Reals, bounds = (0,None))
model.enhanced_china = pyo.Var(domain = pyo.Reals, bounds = (0,None))
model.standard_brazil = pyo.Var(domain = pyo.Reals, bounds = (0,None))
model.enhanced_brazil= pyo.Var(domain = pyo.Reals, bounds = (0,None))

spend_si = model.standard_india 
spend_ei = model.enhanced_india 
spend_sc = model.standard_china 
spend_ec = model.enhanced_china 
spend_sb = model.standard_brazil 
spend_eb = model.enhanced_brazil 

spend_matrix = np.array([[spend_si, spend_sc, spend_sb],
                        [spend_ei, spend_ec, spend_eb]])

In [ ]:
# Calculate the revenues that are a response to the marketing spending
matrix_shape = response_factors.shape
response_revenues = np.zeros(shape = matrix_shape).tolist()
                     
for i in range(matrix_shape[0]):
  for j in range(matrix_shape[1]):
    response_revenues[i][j] =  spend_matrix[i,j][0] * response_factors[i,j]

response_revenues = np.array(response_revenues)

In [ ]:
# objective function 
model.obj = pyo.Objective(expr = response_revenues.sum(), sense=maximize)

# constraints
# Budget limit constraint
model.budget = pyo.Constraint(expr = spend_matrix.sum() <= marketing_budget )

# Response Revenue Constraints
country_responses = [response_revenues[:,country_ind].sum() for country_ind in range(matrix_shape[1])]
product_responses = [response_revenues[product_ind,:].sum() for product_ind in range(matrix_shape[0])]

# Minimum per Country
model.minimum_country_responses = ConstraintList()
for i in range(len(country_responses)):
  model.minimum_country_responses.add( country_responses[i] >=  country_revenue_minima[i])

# Product response constraint
model.enhanced_standard_ratio = pyo.Constraint( expr = product_responses[1] >= product_responses[0]*0.8 )

## Constraint Checks
print(f"The objective function is: \n{model.obj.expr}\n")
print("Constraints")
print("Budgeting constraint")
model.budget.pprint()
print()
print("Minimum responses per Country")
model.minimum_country_responses.pprint()
print()
print("Enhanced/Response Ratio constraint")
model.enhanced_standard_ratio.pprint()

The objective function is: 
0.05*standard_india + 0.04*standard_china + 0.03*standard_brazil + 0.02*enhanced_india + 0.03*enhanced_china + 0.0275*enhanced_brazil

Constraints
Budgeting constraint
budget : Size=1, Index=None, Active=True
    Key  : Lower : Body                                                                                                  : Upper : Active
    None :  -Inf : standard_india + standard_china + standard_brazil + enhanced_india + enhanced_china + enhanced_brazil : 250.0 :   True

Minimum responses per Country
minimum_country_responses : Size=3, Index=minimum_country_responses_index, Active=True
    Key : Lower : Body                                          : Upper : Active
      1 :   3.0 :     0.05*standard_india + 0.02*enhanced_india :  +Inf :   True
      2 :   4.0 :     0.04*standard_china + 0.03*enhanced_china :  +Inf :   True
      3 :   2.0 : 0.03*standard_brazil + 0.0275*enhanced_brazil :  +Inf :   True

Enhanced/Response Ratio constraint
enhanced_

In [ ]:
opt = SolverFactory('glpk')
opt.solve(model)

rows = []
for i in range(matrix_shape[0]):
  product = product_index[i]
  for j in range(matrix_shape[1]):
    country = country_index[j]
    spend = pyo.value(spend_matrix[i,j][0])
    response = pyo.value(response_revenues[i][j])
    rows.append([product,country,spend,response])

import pandas as pd
spend_sf = pd.DataFrame(rows, columns = ['product','country','Spend','Response']).sort_values('country')

#### Marketing Budget Problem Solution

In [ ]:
print("The company should allocate their marketing budget as such:\n")
spend_sf

The company should allocate their marketing budget as such:



,product,country,Spend,Response
2,standard,brazil,0.000000,0.000000
5,enhanced,brazil,72.727273,2.000000
1,standard,china,49.715909,1.988636
4,enhanced,china,67.045455,2.011364
0,standard,india,60.511364,3.025568
3,enhanced,india,0.000000,0.000000


In [ ]:
rev = pyo.value(model.obj)
print("For a total increase in revenue of:")
print(f"£{round(rev,2)}M")

For a total increase in revenue of:
£9.03M


### **Investment Budget: Revenue vs Risk** - Resource Allocation with Low Uncertainty <a class="anchor" id="[](low-uncertainty)"></a>

This problem describes a company's task of allocating their investment budget to various securities, with respect to the predicted returns and risks of default of these securities. 

The objective of the problem is to maximise the returns on these invesetments, while adhering to certain criteria regarding the minimum and maximum percentage of the porfolio a security's investment comprises, as well as the maximum tolerable risk the portfolio can be subject to, as a linearly combined average of the security risks and their proportions of the total of the portfolio. The entire investment budget must be spent, and the amount invested in government bonds must be at least 25% of the total of the other three security investments.

**Problem parameters and variables**
*   Investment budget - £125M
*   Securities - Government Bonds, Municipal Bonds, Corporate Bonds, Consumer Loans
*   Security Expected Annual Returns -  1.5%, 3%, 4.5%, 8%
*   Security Risk of Defaults - 1%, 3%, 7%, 10%
*   Average Risk of portfolio cannot exceed - 6%
*   Minimum investment of any security (proportion of portfolio) - 20%
*   Maximum investment of any security (£M) - 50
*   The investment into Goverment Bonds must be greater than 25% the total of the investments into the other Bonds 

In [ ]:
# Model inputs for revenue, costs, and constraint parameters

# Costs
# Make the price of unobtainable oils sufficiently high such that the amount
# of these oils purchased at this time is forced to be zero for an optmum solution
investment_budget = 125.0
security_returns = [0.015, 0.03, 0.045, 0.08]
security_risks = [0.01, 0.03, 0.07, 0.10]

max_average_risk = 0.06
min_proportion_of_total_per_security = 0.2
max_total_per_security = 50.0
gov_to_bonds_ratio = 0.25 

#### Investment Budget Code

In [ ]:
# general imports
import numpy as np

# import pyomo framework
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [ ]:
# make model variables, of each type(used/bought/stored), for each oil, for each month
num_securities = len(security_returns)
model = pyo.ConcreteModel()

# initiate security investment variables, with minimum and maximum constraints applied
min_total_per_security = min_proportion_of_total_per_security * investment_budget
model.security_investments = pyo.Var(range(num_securities), domain = pyo.Reals, bounds = (min_total_per_security, max_total_per_security))

# Combined Total Invested to be equal to the Investment Budget
model.total_investment_equals_budget = pyo.Constraint(expr = sum([ model.security_investments[i]  for i in range(num_securities)]) == investment_budget )

# Average Risk constriant
model.max_average_risk = pyo.Constraint(expr = sum([ model.security_investments[i] * security_risks[i] for i in range(num_securities)]) <= investment_budget * max_average_risk)

# Min Governement Bond allocation
gov_proportion_of_total_expr =  model.security_investments[0]/gov_to_bonds_ratio + model.security_investments[3] >=  investment_budget
model.min_gov_proportion = pyo.Constraint(expr = gov_proportion_of_total_expr)

# Objective function
model.revenues = sum([ model.security_investments[i] * security_returns[i] for i in range(num_securities)])
model.obj = pyo.Objective(expr = model.revenues, sense=maximize)

opt = SolverFactory('glpk')
opt.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 6.07142857142857, 'Upper bound': 6.07142857142857, 'Number of objectives': 1, 'Number of constraints': 4, 'Number of variables': 5, 'Number of nonzeros': 11, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.013028144836425781}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

#### Investment Budget Solution

In [ ]:
# plot changing % returns based on tolerance of risk (chart 1) and marketing budget (chart 2), both with breakdown of investment per security 

## Stock Levels Problem - Oil Mixing <a class="anchor" id="oil-bullet"></a>
This is a Linear Programming model applied to a stock levels problem, with multiple products/resource variables.

A food manufacturer buys raw vegetable oils and non-vegetable oils, and refines them to produce a margarine called BrandX. When planning the production each month, a plan is made for the following months. Raw oils are bought on the commodity market and, if the price is low, it may be worth buying more oil now and storing it for future use. The planning period starts in January.

Stocks - 
For January no purchases have yet been made, but the manufacturer has 200 tonnes of each raw oil, purchased in previous months, in store. The amount of each raw oil in store should be at this same level at the end of the planning period.

Storage - 
Raw oils, not refined oils, can be stored. Up to 500 tonnes of each raw oil can be stored at any one time, at a cost of £30 per tonne per month.

Refining capacity - 
Vegetable oils must be refined separately from non-vegetable oils. No more than 1000 tonnes of vegetable oil and 800 tonnes of non-vegetable oil can be refined per month.

Hardness - 
The hardnesses of the individual refined oils are given below in array "hardness". The hardness of the blended margarine is found by combining these linearly, in proportion to the quantities used. The hardness of the margarine must lie between 5.6 and 7.4.

Selling price - 
The selling price of the margarine is £2400 per tonne. We need to know the prices of all the oils for immediate delivery and the forecast prices for future months. Prices are forecast according to array "costs_buy", with the row, corresponding to the month, and the column to the oil.

The aim of this model is to maximise profits generated by BrandX, while adhering to the constraints mentioned above, and providing a plan for each month for which of each oil to use, buy, and store in the production of BrandX

In [ ]:
# Model inputs for revenue, costs, and constraint parameters
# general imports
import numpy as np

# Revenue
brandX_revenue_per = 24

# Costs
# Make the price of unobtainable oils sufficiently high such that the amount
# of these oils purchased at this time is forced to be zero for an optmum solution
costs_storage = np.array([0.30,0.30,0.30,0.30,0.30])
unobtainable = brandX_revenue_per*20
costs_buy = np.array([[9.6,  16,         16, 22, 19],
                      [10, unobtainable, 17.8, 20, unobtainable],
                      [10.2, unobtainable, 17.6, 15, unobtainable]])

# physical constraints
hardness = np.array([1.2,3.4,8.0,10.8,8.3])
hardness_min_max = [5.6,7.4]

# storage parameters/constraints
storage_start = np.array([2,2,2,2,2])
storage_end = np.array([2,2,2,2,2])
max_monthly_storage = np.array([5,5,5,5,5])

# Refinement constraints - Separate different types of X (for veg and non-veg, where X = [Veg1, Veg2,..,NonVeg1,NonVeg2,..])
# can be different for each month. rows - months, cols = [veg,non veg]
num_veg = 3
refinement_constraints = np.array([[10,8],
                                   [10,8],
                                   [10,8]])

### Stock Levels Problem - Code

In [ ]:
# import pyomo framework
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory

# make model variables, of each type(used/bought/stored), for each oil, for each month
num_oils = costs_buy.shape[1]
num_months = costs_buy.shape[0]

model = pyo.ConcreteModel()
model.brandX = pyo.Var(range(num_months), bounds = (0,None))
model.stored = pyo.Var(range(num_months-1), range(num_oils), bounds = (0,None))
model.bought = pyo.Var(range(num_months), range(num_oils), bounds = (0,None))
model.used = pyo.Var(range(num_months), range(num_oils), bounds = (0,None))

brandX = model.brandX
stored = model.stored
bought = model.bought
used = model.used

# define objective function
brandX_revenue = sum([brandX[m]*brandX_revenue_per for m in range(num_months)])
cost_storage = sum([ sum([(stored[m,n] * costs_storage[n]) for n in range(num_oils)]) for m in range(num_months-1)])
cost_buy = sum([ sum([(bought[m,n] * costs_buy[m,n] ) for n in range(num_oils)]) for m in range(num_months)])
obj_func = brandX_revenue -cost_storage - cost_buy

model.obj = pyo.Objective(expr = obj_func, sense=maximize)

print("The objective function is:")
print(model.obj.expr)

The objective function is:
24*brandX[0] + 24*brandX[1] + 24*brandX[2] - (0.3*stored[0,0] + 0.3*stored[0,1] + 0.3*stored[0,2] + 0.3*stored[0,3] + 0.3*stored[0,4] + 0.3*stored[1,0] + 0.3*stored[1,1] + 0.3*stored[1,2] + 0.3*stored[1,3] + 0.3*stored[1,4]) - (9.6*bought[0,0] + 16.0*bought[0,1] + 16.0*bought[0,2] + 22.0*bought[0,3] + 19.0*bought[0,4] + 10.0*bought[1,0] + 480.0*bought[1,1] + 17.8*bought[1,2] + 20.0*bought[1,3] + 480.0*bought[1,4] + 10.2*bought[2,0] + 480.0*bought[2,1] + 17.6*bought[2,2] + 15.0*bought[2,3] + 480.0*bought[2,4])


In [ ]:
# Constraints
# add constriant list for each constraint type, then make a sum of the LHS of each constraint to be later added to constraint list 

# sum of used oils == brandX
model.used_oils_sums = ConstraintList()
used_oils_sums = [ sum([used[m,n] for n in range(num_oils)]) for m in range(num_months) ]

# hardness constraints
model.hardness = ConstraintList()
hardness_sum = [sum([used[m,n]*hardness[n] for n in range(num_oils)]) for m in range(num_months)]

# refinement constraints
model.refined = ConstraintList()
refined_veg_sums = [ sum([used[m,n] for n in range(0,num_veg)]) for m in range(num_months) ]
refined_non_veg_sums = [ sum([used[m,n] for n in range(num_veg,num_oils)]) for m in range(num_months) ]
refined_sums = [refined_veg_sums, refined_non_veg_sums]

# Storage constraints
model.max_storage = ConstraintList()
# equation to balance stored prev + bought -used = stored. for all n,m
model.stored_used_bought_equation = ConstraintList()

# loop over months and number of products (oil) and add constraints to constraint lists
for m in range(num_months):
    # sum(used oils) = brandX for each month (m)
    model.used_oils_sums.add(used_oils_sums[m] == brandX[m])
    
    # hardness constraints
    model.hardness.add(hardness_sum[m] >= hardness_min_max[0]*brandX[m])
    model.hardness.add(hardness_sum[m] <= hardness_min_max[1]*brandX[m])
    
    # refinement constraints
    for ref in range(refinement_constraints.shape[1]):
        model.refined.add(refined_sums[ref][m] <= refinement_constraints[m,ref])

    # Storage constraints
    # max storage constraints
    if m < num_months-1:
        for n in range(num_oils):
            model.max_storage.add(stored[m,n] <= max_monthly_storage[n])
            
    # stock balance equation: Stored(prev) + bought - used = stored(current). for all n,m
    for n in range(num_oils):
        if m == 0:
            model.stored_used_bought_equation.add(storage_start[n] + bought[0,n] - used[0,n] == stored[0,n])
        elif m == num_months-1:
            model.stored_used_bought_equation.add(stored[m-1,n] + bought[m,n] - used[m,n] == storage_end[n])
        else:
            model.stored_used_bought_equation.add(stored[m-1,n] + bought[m,n] - used[m,n] == stored[m,n])

print("Constraints (Only first contsraint per constraint type is shown here)"), print()
print("Used Oils = brandX")
model.used_oils_sums[1].pprint()
print()
print("Hardness")
model.hardness[1].pprint()
print()
print("Refinement")
model.refined[1].pprint()
print()
print("Max storage")
model.max_storage[1].pprint()
print()
print("used, bought, stored balance")
model.stored_used_bought_equation[1].pprint()

opt = SolverFactory('glpk')
opt.solve(model)

Constraints (Only first contsraint per constraint type is shown here)

Used Oils = brandX
{Member of used_oils_sums} : Size=3, Index=used_oils_sums_index, Active=True
    Key : Lower : Body                                                                  : Upper : Active
      1 :   0.0 : used[0,0] + used[0,1] + used[0,2] + used[0,3] + used[0,4] - brandX[0] :   0.0 :   True

Hardness
{Member of hardness} : Size=6, Index=hardness_index, Active=True
    Key : Lower : Body                                                                                             : Upper : Active
      1 :  -Inf : 5.6*brandX[0] - (1.2*used[0,0] + 3.4*used[0,1] + 8.0*used[0,2] + 10.8*used[0,3] + 8.3*used[0,4]) :   0.0 :   True

Refinement
{Member of refined} : Size=6, Index=refined_index, Active=True
    Key : Lower : Body                              : Upper : Active
      1 :  -Inf : used[0,0] + used[0,1] + used[0,2] :  10.0 :   True

Max storage
{Member of max_storage} : Size=10, Index=max_storage_index

{'Problem': [{'Name': 'unknown', 'Lower bound': 548.841176470588, 'Upper bound': 548.841176470588, 'Number of objectives': 1, 'Number of constraints': 41, 'Number of variables': 44, 'Number of nonzeros': 130, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.03706812858581543}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

### Stock Levels - Solution

In [ ]:
# Show Plan
print("Optimal buy, use, store plan:\n")
for m in range(num_months):
    vars = [bought, used, stored]
    print(f"In month {m+1}")
    for var in vars:
        if m<num_months-1:
            print(f"{var}:{[np.round(pyo.value(var[m,n]),2) for n in range(num_oils)]}")
        else:
            if var!=stored:
                print(f"{var}:{[np.round(pyo.value(var[m,n]),2) for n in range(num_oils)]}")
    print()
    
profit = pyo.value(model.obj)
print("For a total profit of:")
print(round(profit,2))

Optimal buy, use, store plan:

In month 1
bought:[10.44, 0.0, 3.26, 0.0, 6.0]
used:[7.44, 0.0, 2.56, 2.0, 6.0]
stored:[5.0, 2.0, 2.71, 0.0, 2.0]

In month 2
bought:[4.65, 0.0, 0.0, 8.0, 0.0]
used:[9.65, -0.0, 0.35, 8.0, 0.0]
stored:[0.0, 2.0, 2.35, 0.0, 2.0]

In month 3
bought:[11.65, 0.0, 0.0, 10.0, 0.0]
used:[9.65, 0.0, 0.35, 8.0, 0.0]

For a total profit of:
548.84


# Integer and Mixed Integer Linear Programming <a class="anchor" id="second-bullet"></a>

## Knapsack Problem - Cargo consignment <a class="anchor" id="knapsack-bullet"></a>

A knapsack problem regarding a cargo plane able to carry cargo of given weights and calculated profits. The aim is to maximise the revenue made, while keeping the total cargo weight under the maximum weight capacity of the plane. The cargo items are as follows:
* a consignment of deluxe cars, weighing 30 tonnes, giving a profit of £3000;
* a consignment of small cars, weighing 20 tonnes, giving a profit of £1600;
* several crates of books, weighing 20 tonnes, giving a profit of £500;
* the contents of a house (the owner is moving abroad), weighing 30 tonnes, giving a profit of £1000;
* several crates of canned foodstuffs, weighing 15 tonnes, giving a profit of £600;
* a consignment of timber, weighing 30 tonnes, giving a profit of £2000.

Maximim plane capacity (weight) = 90 tonnes.

Space - 
There is also a space restriction that does not allow the plane to carry both the consignment of deluxe cars and the consignment of small cars in the same flight. 

Upgrade - The cargo company also has the option to upgrade the engines of the plane to allow for 35 tonnes greater weight capacity, caused by the greater thrust. The upgrades will cost £30,000 in total, and the company will only pay for these upgrades if this payment can be made back within two years using the increased profits due to higher cargo capacity. This plane flies twice per month, and the choice of cargo often can arrive at similar total profits for each month.

This is a strictly Integer Linear Programming model, with binary integers corresponding to whether the given cargo items were carried or not, and if the upgrade to the engines was made.

In [ ]:
# Knapsack problem
# Model inputs

# weight
max_weight = 90
cargo_weight = np.array([30,20,20,30,15,40])
upgrade_weight_increase = 35

# revenue/cost
cargo_profit = np.array([3000,1600,500,1000,600,2000])
upgrade_cost = 30000
num_months_to_pay_back = 24
flights_per_month = 2

### Knapsack Problem code

In [ ]:
import numpy as np
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory

# Make Model object
model = pyo.ConcreteModel()

# create variables - Binary
num_items = cargo_weight.shape[0]
model.cargo = pyo.Var(range(num_items), domain=pyo.Binary)
model.upgrade =  pyo.Var(domain=pyo.Binary)
cargo  = model.cargo
upgrade = model.upgrade

# Constraints
# add capacity constraint
capacity_sum = sum([ cargo[item]*cargo_weight[item] for item in range(num_items) ])
model.weight = pyo.Constraint(expr = capacity_sum <= max_weight + upgrade_weight_increase*upgrade)

# add space either/or constraint for car consignments
model.space = pyo.Constraint(expr = model.cargo[0] + model.cargo[1] <=1)

# objective function
upgrade_cost_per_flight = upgrade_cost/(flights_per_month*num_months_to_pay_back)
profit_sum = sum([ cargo[item]*cargo_profit[item] for item in range(num_items) ]) - upgrade_cost_per_flight * upgrade
model.obj = pyo.Objective(expr = profit_sum, sense = maximize)

#solve 
opt = SolverFactory('glpk')
opt.solve(model)

print("Objective Function")
print(model.obj.expr), print()
print("Weight Constraint")
print(model.weight.expr), print()
print("Space Constraint")
print(model.space.expr)

# check feasibility and active constraints
upgrade_made = pyo.value(upgrade)==1
item_values = [ item+1 for item in cargo if (cargo[item].value==1) ]

Objective Function
3000*cargo[0] + 1600*cargo[1] + 500*cargo[2] + 1000*cargo[3] + 600*cargo[4] + 2000*cargo[5] - 625.0*upgrade

Weight Constraint
30*cargo[0] + 20*cargo[1] + 20*cargo[2] + 30*cargo[3] + 15*cargo[4] + 40*cargo[5]  <=  90 + 35*upgrade

Space Constraint
cargo[0] + cargo[1]  <=  1


### Knapsack Problem Solution

In [ ]:
print("Upgrade made = ", upgrade_made), print()
print("Items to take are:")
print(item_values), print()
print("For a profit of:")
print( np.round(pyo.value(model.obj),2 ))

Upgrade made =  True

Items to take are:
[1, 4, 5, 6]

For a profit of:
5975.0


## Goal Programming/Multiple Objectives <a class="anchor" id="goal-bullet"></a>

Sometimes it is useful to have a model that aims to satisfy multiple, potentially conflicting, goals. This is particularly the case when not all of the goals can be satisfied simultaneously (an infeasible solution), and a balance has to be achieved, where some of the goals are achieved and the remaining do not fail too severely.

**A Company with Three Targets** 

A company manufactures three products. The income from sales, the number of person-years required in production, and the cost of raw materials, for each unit of each product, are given in arrays below.

The directors of the company have three targets: to earn an annual income in excess of £100,000; to maintain its workforce at a level of 200; and to keep the bill for raw materials below £50 000 per year. This is a Mixed Integer Programming Problem, with products made being integers, and free and auxillary variables for the objective function being real.

In [ ]:
income_goal = 100
worker_goal = 200
materials_goal = 50


goal_weights = np.array([[1,1],
                         [1,4],
                         [1,1]]) 

product_income = np.array([12,10,8,9])
product_work = np.array([6,5,7,4])
product_materials = np.array([4,3,2,2])

### Goal Programming - Code

In [ ]:
# pyomo multiple goal problem
import numpy as np
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory

# Make Model object
model = pyo.ConcreteModel()
goal_list = [income_goal, worker_goal, materials_goal]

# create variables - integers
num_products = product_income.shape[0]
num_objectives = len(goal_list)
model.products = pyo.Var(range(num_products), domain=pyo.Integers, bounds = (0,None))
products  = model.products

# objective sums
income_sum = sum([product_income[product] * products[product] for product in products ])
worker_sum = sum([product_work[product] * products[product] for product in products ])
materials_sum = sum([product_materials[product] * products[product] for product in products ])
objective_sums = [income_sum,worker_sum,materials_sum]

# create 2 free variables for each auxilliary variable for each objective
model.obj_free_vars = pyo.Var(range(num_objectives*2), domain=pyo.Reals, bounds = (0,None))
model.objective_constraints = pyo.ConstraintList()
for obj in range(num_objectives):
    model.objective_constraints.add(model.obj_free_vars[2*obj]-model.obj_free_vars[2*obj+1] == objective_sums[obj] - goal_list[obj])
    
weights = goal_weights.flatten()
objective_function = sum([model.obj_free_vars[weight] * weights[weight] for weight in model.obj_free_vars])
model.obj = pyo.Objective(expr = objective_function, sense = minimize)
model.objective_constraints.pprint()

#solve 
opt = SolverFactory('glpk')
opt.solve(model)

# check feasibility and active constraints
product_res = np.array([model.products[product].value for product in model.products])

objective_constraints : Size=3, Index=objective_constraints_index, Active=True
    Key : Lower : Body                                                                                                          : Upper : Active
      1 :   0.0 : obj_free_vars[0] - obj_free_vars[1] - (12*products[0] + 10*products[1] + 8*products[2] + 9*products[3] - 100) :   0.0 :   True
      2 :   0.0 :   obj_free_vars[2] - obj_free_vars[3] - (6*products[0] + 5*products[1] + 7*products[2] + 4*products[3] - 200) :   0.0 :   True
      3 :   0.0 :    obj_free_vars[4] - obj_free_vars[5] - (4*products[0] + 3*products[1] + 2*products[2] + 2*products[3] - 50) :   0.0 :   True


### Goal Programming - Solution

In [ ]:
print("Number of products for 1,2,3 made are:")
print(product_res)
income_res = np.dot(product_res, product_income)
worker_res = np.dot(product_res, product_work)
material_res = np.dot(product_res, product_materials)
print("Giving income:", income_res)
print("Keeping workers:", worker_res)
print("Using materials:", material_res)

Number of products for 1,2,3 made are:
[ 0.  0. 28.  1.]
Giving income: 233.0
Keeping workers: 200.0
Using materials: 58.0


## Travelling Salesman Problem <a class="anchor" id="TSP"></a>

The Travelling salesman problem desccribes the attempt to find the minimum total path length of the edges between a collection of nodes, at arbitrary distances from each other, while starting and ending at the same node and only passing through any given node once. Here we have 5 nodes, the distance of edge specifying the journey from node i to j is containd in the matrix "distances" at row i, column j.

In [ ]:
import numpy as np

distances = np.array([[0, 60, 34, 45, 36],
                      [60, 0, 45, 52, 64],
                      [34, 45, 0, 11, 34],
                      [45, 52, 11, 0, 34],
                      [36, 64, 34, 34, 0]])

### Travelling Salesman Problem - Code

In [ ]:
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory

# create variables - integers
num_locations = distances.shape[0]
model = pyo.ConcreteModel()
model.journeys = pyo.Var(range(num_locations), range(num_locations), domain=pyo.Binary)
journeys = model.journeys
    
# add A to B constraints
model.AtoB = pyo.ConstraintList()
model.BtoA = pyo.ConstraintList()

AtoB_sum = [sum([ journeys[i,j] for j in range(num_locations) if i!=j]) for i in range(num_locations)]
BtoA_sum = [sum([ journeys[i,j] for i in range(num_locations) if j!=i]) for j in range(num_locations)]
for journey_sum in range(num_locations):
    model.AtoB.add(AtoB_sum[journey_sum] == 1)
    if journey_sum <num_locations -1:
        model.BtoA.add(BtoA_sum[journey_sum] == 1)


# add auxilliary variables to ensure that each successive journey ends and starts on the same town. E.g. A to B, then B to C
model.successive_aux = pyo.Var(range(0,num_locations), domain = pyo.Integers, bounds = (1,num_locations))
model.successive_constr = pyo.ConstraintList()
successive_aux = model.successive_aux
successive_constr = model.successive_constr

successive_constr.add(successive_aux[0] == 1)
for i in range(num_locations):
    for j in range(1,num_locations):
        if i!=j:
            successive_constr.add(successive_aux[j] - successive_aux[i] >= -(num_locations - 1) + num_locations*journeys[i,j])
          
# objective function
obj_sum = sum([ sum([distances[i,j]*journeys[i,j] for j in range(num_locations) if i!=j]) for i in range(num_locations)])
model.obj = pyo.Objective(expr = obj_sum, sense = minimize)

# Solve 
opt = SolverFactory('glpk')
results = opt.solve(model)

# check feasibility and active constraints
print(results)
route_taken = np.zeros((num_locations,num_locations))
for i in range(num_locations):
    for j in range(num_locations):
        if i!=j:
            route_taken[i,j] = pyo.value(journeys[i,j])


Problem: 
- Name: unknown
  Lower bound: 186.0
  Upper bound: 186.0
  Number of objectives: 1
  Number of constraints: 27
  Number of variables: 26
  Number of nonzeros: 86
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 3
      Number of created subproblems: 3
  Error rc: 0
  Time: 0.03306865692138672
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



### Travelling Salesman Problem - Solution

In [ ]:
print("routes taken:")
route_taken

routes taken:


array([[0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.]])

## Machine Scheduling <a class="anchor" id="MSP"></a>
rows - successive machines \\
columns - products \\
p_ij - time taken for product j to spend on machine i \\

In [ ]:
import numpy as np
# time taken per product j on machine i: Matrix p
p = np.array([
              [10,6,1,2],
              [6,1,14,5],
              [18,8,1,8],
              [12,10,8,4]
])

num_machines = p.shape[0]
num_products = p.shape[1]
t_total = p.sum().sum()

In [ ]:
# create variables - integers
model = pyo.ConcreteModel()
model.time_ends = pyo.Var(range(num_machines), range(num_products), domain=pyo.Reals, bounds = (0,None))
model.total_time = pyo.Var(domain=pyo.Reals, bounds = (0,None))

    
# Total time constraints
model.min_total_time = pyo.ConstraintList()
last_runs = [model.time_ends[(num_machines-1,j)] for j in range(num_products)]
for run in last_runs:
  model.min_total_time.add( run <= model.total_time)
model.min_total_time.pprint(), print()

# successive product_j on a given machine constraints
model.successive_machine_usage = pyo.ConstraintList()
first_runs = [model.time_ends[(0,j)] for j in range(num_products)]
for run in range(len(first_runs)):
  model.successive_machine_usage.add( first_runs[run] >= p[0,run] )

for j in range(0,num_products):
  for run in range(1,num_machines):
    model.successive_machine_usage.add( model.time_ends[(run,j)] >= model.time_ends[(run-1,j)] + p[run,j])
model.successive_machine_usage.pprint(), print()


# not using two products on same machine at the same time
import math
tups = [(i,j,k) for i in range(num_machines) for j in range(num_products-1)  for k in range(j+1, num_products)]
model.one_per_machine = pyo.ConstraintList()
model.on_binaries = pyo.VarList( domain=pyo.Binary, bounds = (0,None))

binary_tup_keys = {}
i = 1
for tup in tups:
  binary_tup_keys[tup] = i
  model.on_binaries.add()
  i+=1

for i in range(num_machines):
  for j in range(num_products-1):
    for k in range(j + 1, num_products):
      model.one_per_machine.add( model.time_ends[(i,j)] - model.time_ends[(i,k)] + p[i,k] <= t_total*model.on_binaries[binary_tup_keys[(i,j,k)]] )
      model.one_per_machine.add( model.time_ends[(i,k)] - model.time_ends[(i,j)] + p[i,j] <= t_total*(1- model.on_binaries[binary_tup_keys[(i,j,k)]]) )
model.one_per_machine.pprint()

min_total_time : Size=4, Index=min_total_time_index, Active=True
    Key : Lower : Body                        : Upper : Active
      1 :  -Inf : time_ends[3,0] - total_time :   0.0 :   True
      2 :  -Inf : time_ends[3,1] - total_time :   0.0 :   True
      3 :  -Inf : time_ends[3,2] - total_time :   0.0 :   True
      4 :  -Inf : time_ends[3,3] - total_time :   0.0 :   True

successive_machine_usage : Size=16, Index=successive_machine_usage_index, Active=True
    Key : Lower : Body                                 : Upper : Active
      1 :  10.0 :                       time_ends[0,0] :  +Inf :   True
      2 :   6.0 :                       time_ends[0,1] :  +Inf :   True
      3 :   1.0 :                       time_ends[0,2] :  +Inf :   True
      4 :   2.0 :                       time_ends[0,3] :  +Inf :   True
      5 :  -Inf :  time_ends[0,0] + 6 - time_ends[1,0] :   0.0 :   True
      6 :  -Inf : time_ends[1,0] + 18 - time_ends[2,0] :   0.0 :   True
      7 :  -Inf : time_ends[2

In [ ]:
# objective function
model.obj = pyo.Objective(expr =  model.total_time, sense = minimize)

# Solve 
opt = SolverFactory('glpk')
results = opt.solve(model)

for i in range(num_machines):
  print(f"For machine: {i}")
  for j in range(num_products):
    start_time = model.time_ends[(i,j)].value - p[i,j]
    print(f"Begin product {j} at {start_time}")
  print()

total_time = [i for i in results['Problem']][0]['Lower bound']
print(f"for a total time of {total_time}")

For machine: 0
Begin product 0 at 7.0
Begin product 1 at 1.0
Begin product 2 at 0.0
Begin product 3 at 17.0

For machine: 1
Begin product 0 at 17.0
Begin product 1 at 15.0
Begin product 2 at 1.0
Begin product 3 at 23.0

For machine: 2
Begin product 0 at 24.0
Begin product 1 at 16.0
Begin product 2 at 15.0
Begin product 3 at 42.0

For machine: 3
Begin product 0 at 42.0
Begin product 1 at 24.0
Begin product 2 at 34.0
Begin product 3 at 54.0

for a total time of 58.0
